In [1]:
import sys, os 
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import glob
import numpy as np
import pandas as pd
import gezi
from gezi import tqdm

2021-10-14 16:58:12.241948: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [11]:
import numpy as np
import gezi
from gezi import Vocab
import json
model_name = 'macbert'
root = '../working/offline/35/'
model_dir = f'{root}/{model_name}'
fold = 0
vocab = Vocab('../input/pairwise/vid.vocab', 0)
print(vocab.size())
file = f'{model_dir}/pairwise/{fold}/valid.json'
with open(file) as f:
 vid_embedding = json.load(f)
emb_dim = 256
embs = np.zeros((vocab.size(), emb_dim))
print(np.mean(embs))
for vid, emb in vid_embedding.items():
  embs[vocab.id(vid)] = np.asarray(emb)
print(np.mean(embs))
np.save(f'{model_dir}/pairwise/{fold}/valid.npy')

63614
0.0
0.003945413626970014


In [12]:
from transformers import TFAutoModel, TFAutoModelForMaskedLM

In [14]:
x = TFAutoModel.from_pretrained('../input/pretrain/word/0/tiny/bert')

Some layers from the model checkpoint at ../input/pretrain/word/0/tiny/bert were not used when initializing TFAlbertModel: ['predictions', 'tf_albert_for_masked_lm/albert/encoder/albert_layer_groups_._0/albert_layers_._0/attention/query/bias:0', 'tf_albert_for_masked_lm/albert/encoder/albert_layer_groups_._0/albert_layers_._0/attention/LayerNorm/gamma:0', 'tf_albert_for_masked_lm/albert/encoder/albert_layer_groups_._0/albert_layers_._0/attention/query/kernel:0', 'tf_albert_for_masked_lm/albert/embeddings/LayerNorm/beta:0', 'tf_albert_for_masked_lm/albert/encoder/albert_layer_groups_._0/albert_layers_._0/attention/dense/bias:0', 'tf_albert_for_masked_lm/albert/encoder/albert_layer_groups_._0/albert_layers_._0/attention/key/kernel:0', 'tf_albert_for_masked_lm/albert/encoder/albert_layer_groups_._0/albert_layers_._0/full_layer_layer_norm/gamma:0', 'tf_albert_for_masked_lm/albert/embeddings/LayerNorm/gamma:0', 'tf_albert_for_masked_lm/albert/encoder/albert_layer_groups_._0/albert_layers_._

In [18]:
x.layers[0]

In [28]:
x = TFAutoModel.from_pretrained('../input/pretrain/0/bert-base-chinese/bert')

Some layers from the model checkpoint at ../input/pretrain/0/bert-base-chinese/bert were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at ../input/pretrain/0/bert-base-chinese/bert and are newly initialized: ['bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
x.layers

In [24]:
x.layers[0]

In [7]:
embs

array([[ 0.04416986,  0.03421553, -0.03631344, ..., -0.03308839,
        -0.00273736, -0.0053576 ],
       [ 0.00611336, -0.03274676, -0.03784559, ...,  0.02179161,
        -0.04844187,  0.0049248 ],
       [ 0.04062833,  0.02554473,  0.01683699, ...,  0.02588887,
        -0.00560684, -0.01232914],
       ...,
       [-0.03317818,  0.02776423, -0.02806441, ...,  0.02959499,
        -0.04396443, -0.02489318],
       [ 0.0191858 , -0.0322538 , -0.02458396, ..., -0.03428076,
         0.02351121, -0.03878129],
       [ 0.04030947, -0.04770916, -0.0461725 , ..., -0.04363672,
        -0.01946393, -0.01658311]])

In [ ]:
label = pd.read_csv('../input/pairwise/label.tsv', names=['vid1', 'vid2', 'relevance'], sep='\t')

In [ ]:
label

In [ ]:
len(set(label.vid1))

In [ ]:
len(set(label.vid2))

In [ ]:
len(set(label.vid1) | set(label.vid2))

In [ ]:
!wc -l ../input/pairwise/ids.csv

In [ ]:
len(set(label.vid1) & set(label.vid2))

In [ ]:
list(set(label.vid1) & set(label.vid2))[:10]

In [ ]:
label[label.vid1 == 1282350513519752622]

In [ ]:
label[label.vid2 == 1282350513519752622]

In [ ]:
FOLDS = 5

In [ ]:
SEED = 521

In [ ]:
intersect_vids = set(label.vid1) & set(label.vid2)

In [ ]:
np.random.seed(SEED)

In [ ]:
vids = list(set(label.vid1))
np.random.shuffle(vids)

In [ ]:
vids = [x for x in vids if x not in intersect_vids]

In [ ]:
vids_list = np.array_split(vids, FOLDS)

In [ ]:
[len(x) for x in vids_list]

In [ ]:
# vids_list[-1] = np.asarray(list(vids_list[-1]) + list(intersect_vids))

In [ ]:
[len(x) for x in vids_list]

In [ ]:
vids_list

In [ ]:
label2 = label[~label.vid1.isin(intersect_vids) & ~label.vid2.isin(intersect_vids)]

In [ ]:
label2

In [ ]:
label3 = label[label.vid1.isin(intersect_vids) | label.vid2.isin(intersect_vids)]

In [ ]:
label3

In [ ]:
# label2 = label

In [ ]:
for i in range(FOLDS):
  d = label2[label2.vid1.isin(set(vids_list[i]))]
  print(len(d))
  d.to_csv(f'../input/pairwise/label_{i}.csv', index=False)

In [ ]:
label3.to_csv(f'../input/pairwise/label_{FOLDS}.csv', index=False)

In [ ]:
len(label)

In [ ]:
len(label2)

In [ ]:
len(vids_list[0])

In [ ]:
label2[label2.vid1.isin(vids_list[0])]

In [ ]:
len(set(label2.vid1))

In [ ]:
vids_list[4]

In [ ]:
ds = {}
for i in range(FOLDS):
  ds[i] = pd.read_csv(f'../input/pairwise/label_{i}.csv')

In [ ]:
def intersect(x, y):
  return len(set(list(x.vid1.values) + list(x.vid2.values)) & set(list(y.vid1.values) + list(y.vid2.values)))

In [ ]:
for i in range(FOLDS):
  for j in range(FOLDS):
      if j > i:
        print(i, j, intersect(ds[i], ds[j]))

In [ ]:
intersect(pd.concat([ds[1],ds[2],ds[3],ds[4]]), ds[0])

In [ ]:
dt = pd.concat([ds[1],ds[2],ds[3],ds[4]])

In [ ]:
len(set(dt.vid1 | dt.vid2))

In [ ]:
len(ds[4])

In [ ]:
ds[0][ds[0].vid1.isin(set(dt.vid1))]

In [ ]:
ds[0][ds[0].vid2.isin(set(dt.vid2))]

In [ ]:
dt

In [ ]:
ds[4]